In [1]:
from sqlalchemy import create_engine
from key import DB_USER, DB_PASSWORD, DB_ENDPOINT

In [84]:
#assign import secret values to varibles
user = DB_USER
pwd = DB_PASSWORD
endpoint = DB_ENDPOINT

#create url for engin
dialect = f"mysql+pymysql://{user}:{pwd}@{endpoint}"

#create engin
engine = create_engine(dialect, echo = True, future=True)

In [6]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2021-05-07 09:04:23,861 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-07 09:04:23,862 INFO sqlalchemy.engine.Engine select 'hello world'
2021-05-07 09:04:23,863 INFO sqlalchemy.engine.Engine [generated in 0.00239s] {}
[('hello world',)]
2021-05-07 09:04:23,893 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
with engine.connect() as conn:
    conn.execute(
        text("USE RockClimbingGyms")
    )
    conn.commit()
    

2021-05-07 09:28:49,381 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-07 09:28:49,382 INFO sqlalchemy.engine.Engine USE RockClimbingGyms
2021-05-07 09:28:49,383 INFO sqlalchemy.engine.Engine [generated in 0.00165s] {}
2021-05-07 09:28:49,416 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
with engine.connect() as conn:
    conn.execute(
        text("CREATE TABLE some_table (x int, y int)")
    )
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
             [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
    )
    conn.commit()

2021-05-07 09:28:56,666 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-07 09:28:56,667 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2021-05-07 09:28:56,667 INFO sqlalchemy.engine.Engine [cached since 250s ago] {}
2021-05-07 09:28:56,805 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2021-05-07 09:28:56,806 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ({'x': 1, 'y': 1}, {'x': 2, 'y': 4})
2021-05-07 09:28:56,839 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
             [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )

2021-05-07 09:36:34,620 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-07 09:36:34,621 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2021-05-07 09:36:34,622 INFO sqlalchemy.engine.Engine [cached since 457.8s ago] ({'x': 6, 'y': 8}, {'x': 9, 'y': 10})
2021-05-07 09:36:34,652 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
from sqlalchemy.orm import Session

stmp = text("UPDATE some_table SET y=:y WHERE x=:x").bindparams(x=9, y=11)
with Session(engine) as session:
    results = session.execute(stmp)
    session.commit()

2021-05-07 11:30:15,788 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-07 11:30:15,790 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=%(y)s WHERE x=%(x)s
2021-05-07 11:30:15,791 INFO sqlalchemy.engine.Engine [generated in 0.00105s] {'y': 11, 'x': 9}
2021-05-07 11:30:15,823 INFO sqlalchemy.engine.Engine COMMIT


In [46]:
from sqlalchemy import MetaData, Table, Column, Integer, String

metadata = MetaData()
user_table = Table(
    "user_account",
    metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String(80))
)

In [47]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [48]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable=False),
    Column('email_address', String(80), nullable=False)
)

In [49]:
metadata.create_all(engine)

2021-05-07 15:16:50,062 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-07 15:16:50,063 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-05-07 15:16:50,064 INFO sqlalchemy.engine.Engine [cached since 265.3s ago] {'table_schema': 'None', 'table_name': 'user_account'}
2021-05-07 15:16:50,093 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-05-07 15:16:50,094 INFO sqlalchemy.engine.Engine [cached since 265.3s ago] {'table_schema': 'None', 'table_name': 'address'}
2021-05-07 15:16:50,126 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(30), 
	fullname VARCHAR(80), 
	PRIMARY KEY (id)
)


2021-05-07 15:16:50,127 INFO sqlalchemy.engine.Engine [no key 0.00072s] {}
2021-05-07 15:16:50,178 INFO sqlalchemy.engine.Engine 
CREAT

In [51]:
from sqlalchemy.orm import registry

mapper_registry = registry()
mapper_registry.metadata

Base = mapper_registry.generate_base()

In [53]:
from sqlalchemy.orm import relationship

class User(Base):
    __table__ = user_table

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"

class Address(Base):
    __table__ = address_table

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"


In [60]:
some_table = Table("some_table", metadata, autoload_with=engine)

2021-05-09 14:34:47,377 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-09 14:34:47,378 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `some_table`
2021-05-09 14:34:47,380 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-05-09 14:34:47,432 INFO sqlalchemy.engine.Engine ROLLBACK


In [65]:
from sqlalchemy import insert
stmt = insert(user_table).values(name='spongebob', fullname='spongebob squrepants')
compiled = stmt.compile()
print(compiled.params)

with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()
    print(result.inserted_primary_key)
    

{'name': 'spongebob', 'fullname': 'spongebob squrepants'}
2021-05-09 17:43:35,622 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-09 17:43:35,624 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name)s, %(fullname)s)
2021-05-09 17:43:35,627 INFO sqlalchemy.engine.Engine [cached since 46.37s ago] {'name': 'spongebob', 'fullname': 'spongebob squrepants'}
2021-05-09 17:43:35,680 INFO sqlalchemy.engine.Engine COMMIT
(1,)


In [67]:
from sqlalchemy import select

stmt = select(user_table).where(user_table.c.name == 'spongebob')
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [74]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2021-05-09 18:23:28,814 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-09 18:23:28,816 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s
2021-05-09 18:23:28,819 INFO sqlalchemy.engine.Engine [generated in 0.00488s] {'name_1': 'spongebob'}
(1, 'spongebob', 'spongebob squrepants')
2021-05-09 18:23:28,862 INFO sqlalchemy.engine.Engine ROLLBACK


In [75]:
stmt = select(User).where(User.name == 'spongebob')

with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2021-05-09 18:28:20,226 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-09 18:28:20,249 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s
2021-05-09 18:28:20,252 INFO sqlalchemy.engine.Engine [generated in 0.00288s] {'name_1': 'spongebob'}
(User('spongebob', 'spongebob squrepants'),)
2021-05-09 18:28:20,300 INFO sqlalchemy.engine.Engine ROLLBACK


In [86]:
session = Session(engine)

In [87]:
squidward = User(name='squidward', fullname='squidward tentacles')
krabs = User(name='krabs', fullname='Eugene H. Krabs')

In [88]:
session.add(squidward)

In [89]:
session.new

IdentitySet([User('squidward', 'squidward tentacles')])

In [90]:
session.add(krabs)

In [91]:
session.new

IdentitySet([User('squidward', 'squidward tentacles'), User('krabs', 'Eugene H. Krabs')])

In [92]:
session.flush()

2021-05-09 19:42:42,745 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-09 19:42:42,749 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name)s, %(fullname)s)
2021-05-09 19:42:42,751 INFO sqlalchemy.engine.Engine [generated in 0.00211s] {'name': 'squidward', 'fullname': 'squidward tentacles'}
2021-05-09 19:42:42,796 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name)s, %(fullname)s)
2021-05-09 19:42:42,798 INFO sqlalchemy.engine.Engine [cached since 0.04874s ago] {'name': 'krabs', 'fullname': 'Eugene H. Krabs'}


In [93]:
squidward.id

4

In [94]:
session.commit()

2021-05-09 19:44:48,314 INFO sqlalchemy.engine.Engine COMMIT
